# d) Setup Environmental Flow Requirements

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

Based on the environmental flow bands, the methodology adopted for the energy modelling is as below:

- The flow on the 8th, 18th and 28th day of each month is identified. 
- The equivalent percentile of this flow based on flow exceedance curves is calculated for that day from the 92 years of data.
- This percentile is used to decide the environmental flow release for the succeeding ~10 day periods of 1st to 10th, 11th to 20th and 21st to end of the month.
- Environmental flow in this band and month is released in the respective channels.

In this notebook the exceedance percentiles for each day of the historic flow series are calculated. These will later be used to lookup the required EWR values for each channel depending on the Environmental Assurance Category selected for the channel. 

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ngonye_flow_daily.csv        | Notebook: c_flow_data | Daily Flow series              |



## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_flow_daily_ewrx.csv  | Daily flow series for Ngonye with EWR Exceedance values  |


## Parameters

In [21]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [22]:
import numpy as np
import pandas as pd

## Load Daily Data

In [23]:
daily=pd.read_csv(output_data + "ngonye_flow_daily.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)

## Build FDCs
Build FDCs for each of the 'measurement days' - 8th, 18th, 28th of each month

In [24]:
cols=['9-28','10-8','10-18','10-28','11-8','11-18','11-28','12-8','12-18','12-28',
      '1-8','1-18','1-28','2-8','2-18','2-28','3-8','3-18','3-28','4-8','4-18',
      '4-28','5-8','5-18','5-28','6-8','6-18','6-28','7-8','7-18','7-28','8-8','8-18','8-28','9-8','9-18']
ewr_fdcs=pd.DataFrame(columns=cols,index=np.arange(0,1.01,0.01))

for column in ewr_fdcs:   
    (month,day)=column.split('-')
    ewr_fdcs[column]=np.percentile(daily[(daily['Month']==int(month)) & (daily['Day']==int(day))]['Flow'] ,(1-ewr_fdcs.index)*100,interpolation='linear')
    
ewr_fdcs.tail(5)

,9-28,10-8,10-18,10-28,11-8,11-18,11-28,12-8,12-18,12-28,...,6-18,6-28,7-8,7-18,7-28,8-8,8-18,8-28,9-8,9-18
0.96,179.747804,165.630893,148.811132,154.702178,154.691402,173.010587,200.601842,241.345489,276.423297,339.975662,...,416.142297,367.981714,341.414602,321.900903,291.013915,268.520799,234.817519,220.790945,202.743568,192.440410
0.97,169.881107,148.297489,138.940445,146.795479,148.665417,170.439890,194.443204,232.452444,265.750305,320.952985,...,387.785736,345.850477,327.753898,307.087715,284.766704,258.896041,230.438584,212.818697,199.804625,184.989573
0.98,152.677266,125.878777,116.912047,124.123539,142.747171,161.423169,190.505029,222.170543,260.335273,275.168456,...,339.881910,307.492983,287.681326,263.249807,259.712637,236.307631,219.825566,202.894567,190.831594,177.422958
0.99,138.797377,122.972481,112.155549,115.931495,138.622713,144.652165,186.563703,195.233840,235.930137,264.667848,...,325.272624,289.747717,269.319967,252.893518,231.400287,218.047310,197.461671,187.330502,165.998093,153.817529
1.00,135.364322,111.133124,111.133124,107.211215,133.181027,141.269277,182.967257,192.534369,235.930137,249.436217,...,315.726484,275.186447,259.369380,238.490295,224.058948,207.912046,195.867340,178.800013,160.370675,144.946329


## Functions

In [25]:
#Gives the measure date (proceeding 8th, 18th or 28th) for the given date
def measureDate(obsDate):
    if obsDate.day <= 10:
        if obsDate.month==1:
            mn=13
            yr=obsDate.year-1
        else:
            mn=obsDate.month
            yr=obsDate.year
        return pd.Timestamp(yr, mn-1, 28)
    elif obsDate.day <=20:
        return pd.Timestamp(obsDate.year, obsDate.month, 8)
    else:
        return pd.Timestamp(obsDate.year, obsDate.month, 18)

#If it is a measure date (8th, 18th or 28th) then returns the ID of an FDC for that date. Otherwise False.
def isMeasureDay(obsDate):
    if obsDate.day==8 or obsDate.day==18 or obsDate.day==28:
        return str(obsDate.month) + '-' + str(obsDate.day) 
    else:
        return False
    
#Given an FDC ID and flow looks up exceedance
def lookupExceedance(measureDay,flow):
    return ewr_fdcs[ewr_fdcs[measureDay]<=flow].index.min()

## Exceedance Lookup
Build a table of measurement days and lookup the exceedance value for these from the FDCs already built

In [26]:
daily['dDate']=daily.index
daily['MeasureDay']=daily.apply(lambda x: isMeasureDay(x['dDate']),axis=1)
ewr_measure_days=pd.DataFrame(daily.loc[daily['MeasureDay']!=False]['MeasureDay'])
ewr_measure_days['Flow']=daily['Flow']
ewr_measure_days['Exceedance']=ewr_measure_days.apply(lambda x: lookupExceedance(x['MeasureDay'],x['Flow']),axis=1)
ewr_measure_days.tail(3)

,MeasureDay,Flow,Exceedance
Date,,,
2017-09-08,9-8,240.615078,0.81
2017-09-18,9-18,216.320970,0.82
2017-09-28,9-28,203.017496,0.83


## EWR Measure Dates
Mark the dates on the daily timeseries where the EWR will be re-measured (8th, 18th, 28th of each month) 

In [27]:
daily['dDate']=daily.index
daily['EWRMeasureDate']=daily.apply(lambda x: measureDate(x['dDate']),axis=1)

## Daily EWR Exceedance
Lookup the EWR exceedance values for each day of the daily series based on the measure date

In [28]:
daily['EWRRefExceedance']=daily.join(ewr_measure_days,on='EWRMeasureDate',lsuffix='_l')['Exceedance']

In [29]:
daily=daily.drop(['MeasureDay','EWRMeasureDate','dDate'],axis=1)
daily=daily.fillna(0.5)
daily.tail(3)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,WaterWeek,Volume,EWRRefExceedance
Date,,,,,,,,,,,,,,,
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,52,0.017541,0.82
2017-09-29,2017-10-10,204.0,0.995184,203.017496,0.950,2017,9,29,2017.09,2016,12,364,52,0.017541,0.82
2017-09-30,2017-10-11,201.0,0.986217,198.229576,0.958,2017,9,30,2017.09,2016,12,365,53,0.017127,0.82


## Save

In [30]:
daily.to_csv(output_data + 'ngonye_flow_daily_ewrx.csv')